# Participle Research

## 1. Data Extract

In [4]:
from test import test
import sys
sys.path.append('..')
import action

handle = open("../Process_type/examples.txt", "r")
text = handle.read()
handle.close()
list_, N = [0], 0
for i in range(len(text)):
    if text[i] == '.':
        last = i
    N += 1
    if N > 10000:
        list_.append(last)
        N = 0
root_list = []
for i in range(len(list_) - 1):
    root_list = root_list + action.construct_tree(text[list_[i] : list_[i + 1]])

list_ = test(root_list)
print(len(list_))

165


In [19]:
def sub(name, list_):
    if name in list_:
        return 1
    return 0

dict_1, dict_2  = {}, {}
for i in list_:
    for j in i[3]:
        dict_[j[1]] = []
        dict_[j[0].value.postag] = []
        dict_1[j[1]] = []
        dict_2[j[0].value.postag] = []

dict_['Sentence'], dict_['Lemma'], dict_['Token'] = [], [], []

for i in list_:
    dict_['Sentence'].append(i[0])
    dict_['Lemma'].append(i[1])
    dict_['Token'].append(i[2])
    for j in dict_1.keys():
        dict_[j].append(sub(j, [k[1] for k in i[3]]))
    for j in dict_2.keys():
        dict_[j].append(sub(j, [k[0].value.postag for k in i[3]]))
import pandas as pd
df = pd.DataFrame(data = dict_)
df.to_csv('Part.csv', index = False, encoding = 'utf')